## Отправка

In [ ]:
!pip install whatsapp-chat-exporter

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import webbrowser
from selenium import webdriver
import urllib
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from tqdm import notebook

def element_presence(by, xpath, time):
    element_present = EC.presence_of_element_located((By.XPATH, xpath))
    WebDriverWait(driver, time).until(element_present)
    
def send_message(url):
    driver.get(url)
    time.sleep(2)
    element_presence(By.XPATH, '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div/p', 40)
    msg_box = driver.find_element(By.XPATH, '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div/p')
    msg_box.send_keys('\n')
    time.sleep(1)
    
def prepare_msg(dataframe, name_col, phone_col):
    file = dataframe[[name_col, phone_col]]
    with open('Template.txt', 'r', encoding='utf-8') as file:
    base_msg = file.read().replace('"', '')
    
    #base_msg = """
    #*Здравствуйте {}*!
    #Проверка связи!
    #Спасибо!
    #"""
    
    base_url = 'https://web.whatsapp.com/send?phone={}&text={}'
    for i,j in notebook.tqdm(file.iterrows()):
        phone_no = j[phone_col]
        Name = j[name_col].title()
        msg = urllib.parse.quote(base_msg.format(Name))
        url_msg = base_url.format(phone_no, msg)
        send_message(url_msg)
        
chrome_options = Options()
chrome_options.add_argument("--user-data-dir-Session")
chrome_options.add_argument("--profile-directory=Default")

import pandas as pd
#dummy2=pd.DataFrame({'Name': ['Глеб', 'Марк'], 'Phone': ['+79045174867', '+79817703936']})
dummy2 = pd.read_excel('Whatsapp_Input.xlsx')

import time
time_start = time.time()
driver = webdriver.Chrome(options=chrome_options)
prepare_msg(dummy2, 'Name', 'Phone')
time_end = time.time()

print(f"Отправлено {len(dummy2)} сообщений")
print(f"Время затрачено {round(time_end - time_start)} сек")

## Экспорт резервной копии

wtsexporter -i -b "C:\Users\*Имя пользователя*\AppData\Roaming\Apple Computer\MobileSync\Backup\*Код копии*" -o "\Whatsapp_export"

## Анализ и вывод

In [1]:
from html.parser import HTMLParser
import os, io, re

class MyHTMLParserHP(HTMLParser):
    def handle_data(self, data):
        if not re.match(r'^\s*$', data):
            result.append(data)

result_dict = {'Phone': [], 'Feedback': []}  
            
for filename in os.listdir('Whatsapp_export'):
    try:
        result = []
        parser = MyHTMLParserHP()
        with io.open('Whatsapp_export/'+filename, encoding = 'utf-8') as file:
            for line in file:
                parser.feed(line)
                
        time_index = []
        for index, line in enumerate(result):
            if (re.match("^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$", line)):
                time_index.append(index)
        time_index

        history = []
        for index in time_index:
            if result[index+1].startswith('\t'):
                history.append(result[index+1].replace('\t', '').replace('\n', ''))
                
        if 'Проверка связи!' in result:
            result_dict['Phone'].append(filename[:filename.index('-')])
            result_dict['Feedback'].append(history)
    except Exception as e:
        print(e)
        pass
    
df_final = pd.DataFrame(result_dict)
df_final['Feedback'] = df_final['Feedback'].apply(lambda x: ", ".join(x))
df_final.to_excel('Whatsapp_Output.xlsx')
df_final

[Errno 13] Permission denied: '123/Message'
